# 🌴 Asistente Turístico de Tenerife - RAG Demo

## 1. Configuración del Entorno
Cargamos las librerías necesarias y verificamos la API Key.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ Key cargada: {api_key[:5]}...")
else:
    print("❌ Error de Key")

## 2. Validación de Conectividad con OpenAI
Probamos que la llave funcione y el modelo responda.

In [ ]:
from langchain_openai import ChatOpenAI

# Inicializamos el modelo
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Prueba 'Holística'
try:
    response = llm.invoke("Di 'Hola Tenerife' si me escuchas.")
    print(f"🤖 Respuesta Modelo: {response.content}")
except Exception as e:
    print(f"❌ Error de Conexión: {e}")

## 3. Fase 2: Ingesta y Vector Store 📚
Subimos el PDF `TENERIFE.pdf` a OpenAI para crear un Vector Store gestionado.

In [ ]:
import os
import openai
from openai import OpenAI
from pathlib import Path

# DEBUG: Verificar entorno
print(f"📂 CWD: {os.getcwd()}")
print(f"📦 OpenAI Version: {openai.__version__}")

client = OpenAI()

# Path de datos
pdf_path = Path("../data/raw/TENERIFE.pdf")

if not pdf_path.exists():
    print(f"❌ No encontrado: {pdf_path}")
else:
    print(f"✅ Archivo encontrado: {pdf_path}")

    # Crear Vector Store (Compatible con V1 y V2)
    try:
        # En algunas versiones V2, vector_stores esta directamente en beta
        if hasattr(client, 'beta') and hasattr(client.beta, 'vector_stores'):
            vs_manager = client.beta.vector_stores
        else:
            raise AttributeError("La librería OpenAI instalada no expone 'client.beta.vector_stores'. Intenta: %pip install -U openai")
            
        vector_store = vs_manager.create(name="Tenerife Guide Store")
        print(f"📦 Vector Store creado: {vector_store.id}")
        
        # Subir archivos
        with open(pdf_path, "rb") as f:
            file_batch = vs_manager.file_batches.upload_and_poll(
                vector_store_id=vector_store.id, files=[f]
            )
        print(f"📄 Estado de carga: {file_batch.status}")
        print(f"🔢 Archivos procesados: {file_batch.file_counts}")
            
    except Exception as e:
        print(f"❌ Error Critical: {e}")

## 4. Fase 3: Configuración del Agente RAG 🤖
Configuramos el agente con capacidad de usar el Vector Store para responder preguntas sobre Tenerife.
**Nota**: Asegúrate de haber ejecutado la Fase 2 para tener la variable `vector_store` disponible.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from openai_agents.agents import Agent, Runner
from openai_agents.tools import FileSearchTool

# Intentamos obtener el ID del store creado en la celda anterior.
# Si se reinició el kernel, se puede usar el ID hardcodeado de la ejecución previa.
try:
    vector_store_id = vector_store.id
except NameError:
    print("⚠️ 'vector_store' no está en memoria. Usando ID precargado (asegúrate que sea válido).")
    vector_store_id = "vs_69661c161c5881919eadf77744958c70"

# 1. Definir la herramienta de búsqueda de archivos
file_search_tool = FileSearchTool(vector_store_id=vector_store_id)

# 2. Instrucciones del Agente (System Prompt)
INSTRUCTIONS = """
Eres un experto guía turístico de Tenerife.
Usa la herramienta file_search para encontrar información en la guía PDF adjunta.
Si la información no está en el PDF, dilo claramente. No inventes respuestas.
Responde siempre en español, con un tono amable y entusiasta.
"""

# 3. Crear el Agente
agent = Agent(
    name="TenerifeGuide",
    model="gpt-3.5-turbo",
    instructions=INSTRUCTIONS,
    tools=[file_search_tool]
)

print(f"🤖 Agente '{agent.name}' configurado satisfactoriamente.")

### Prueba Rápida del Agente
Verificamos que el agente pueda recuperar información del PDF.

In [ ]:
async def test_agent():
    result = await Runner.run(agent, "¿Cuáles son las mejores playas del sur de Tenerife?")
    print(f"❓ Pregunta: ¿Cuáles son las mejores playas del sur de Tenerife?")
    print(f"💡 Respuesta: {result.final_response}")

await test_agent()